In [1]:
import sys
print(sys.executable)


c:\Users\marce\OneDrive - mtegovbr\0001_NOVA_PASTA\DESENVOLVIMENTO_PESSOAL\001_mba_ciencia_dados_enap\007_ia_generativa_contexto_adm_pub\chatbot-pronaf\venv3.13\Scripts\python.exe


In [2]:
try:
    import chromadb
    print("chromadb importado com sucesso:", chromadb.__version__)
except Exception as e:
    print("Erro ao importar chromadb:", e)

chromadb importado com sucesso: 1.0.0


In [3]:
# Biblioteca de código para realizar a busca de informações em um PDF usando LangChain e ChromaDB
# Criação do Banco de Dados ChromaDB com os embeddings do HuggingFace
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


# Criação do RAG (Retrieval-Augmented Generation) com o modelo de linguagem OpenAI
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
# importar: name 'RunnableLambda' is not defined
from langchain_core.runnables import RunnableLambda

# Chave da API do OpenAI
import streamlit as st
import openai


In [4]:

# Carregar variáveis de ambiente
load_dotenv()

# Configurar chave da API OpenAI
try:
    #OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
    openai.api_key = st.secrets["openai_api_key"]
    if not openai.api_key:
        raise ValueError("'openai.api_key' não encontrada no ambiente.")
except Exception as e:
    raise RuntimeError(f"Erro ao carregar a chave da API OpenAI: {e}")


In [5]:
# Carregar o documento PDF
try:
    pdf_path = r"data\data-rag\manual_credito_rural_20250406.pdf"
    dados = PyPDFLoader(pdf_path).load()
except Exception as e:
    raise RuntimeError(f"Erro ao carregar o documento PDF: {e}")


In [6]:
### Limitar o número de páginas a serem processadas
# dados = dados[:20]  # Limitar a 10 páginas para evitar sobrecarga de memória
# dados

In [7]:
# Dividir o texto em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
textos = text_splitter.split_documents(dados)
textos

[Document(metadata={'producer': 'iText® 5.4.4 ©2000-2013 1T3XT BVBA (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2025-04-02T14:43:45-03:00', 'moddate': '2025-04-02T14:43:45-03:00', 'source': 'data\\data-rag\\manual_credito_rural_20250406.pdf', 'total_pages': 361, 'page': 0, 'page_label': '1'}, page_content='1\nAtualização MCR nº 694, de 7 de maio de 2021 \nMANUAL DE CRÉDITO RURAL (MCR) \nCodificação \nInstruções   \n \n1 - O Manual de Crédito Rural (MCR)  codifica as normas aprovadas pelo Conselho Monetário Nacional (CMN)  \ne aquelas divulgadas pelo Banco Central do Brasil relativas ao crédito rural, às quais  devem subordinar-se os \nbeneficiários e as instituições financeiras que operam n o Sistema Nacional de Crédito Rural (SNCR) , sem'),
 Document(metadata={'producer': 'iText® 5.4.4 ©2000-2013 1T3XT BVBA (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2025-04-02T14:43:45-03:00', 'moddate': '2025-04-02T14:43:45-03:00', 'source': 'data\\data-rag\\manual_credito_rural_

In [8]:
# Criar embeddings usando HuggingFace
embedding_engine = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [9]:
import os  # Certifique-se de importar o módulo os

# Criar o diretório de persistência se não existir
os.makedirs(r"data\data-rag\persist_directory", exist_ok=True)

# Criar e persistir o banco de dados vetorial
try:
    vector_db = Chroma.from_documents(
        textos, 
        embedding_engine, 
        persist_directory=r"data\data-rag\persist_directory"
    )

except Exception as e:
    raise RuntimeError(f"Erro ao criar o banco de dados vetorial: {e}")


In [10]:
### Teste
#print(vector_db) 
docs_teste = vector_db.similarity_search("pronaf", k=2)
for i, doc in enumerate(docs_teste, start=1):
    print(f"Documento {i}:\n{doc.page_content}\n")


Documento 1:
Declaração de Aptidão ao Pronaf (DAP) 
ou do documento Cadastro Nacional da 
Agricultura Familiar do Pronaf (CAF -
Pronaf) ao agente financeiro, fica 
habilitada a novos créditos nesse grupo, 
nas mesmas condições desta linha de 
2 - demais beneficiários: UFPA R$4.000,00

Documento 2:
Pronaf) e 2.1.00.20-7 (Subexigibilidade Pronaf - Total), sendo que só ocorrerá excesso quando o valor do 
primeiro código for maior que o do segundo. 
 
5.1.31.00-8 Deficiência referente à Subexigibilidade Pronamp. 
O valor desse código indica a deficiência de aplicação em crédito rural referente à Subexigibilidade 
Pronamp, dada pela  diferença entre os códigos 2.1.00.30 -0 (Subexigibilidade Pronamp - Total) e



In [11]:
# Configurar o modelo LLM
llm = ChatOpenAI(openai_api_key=openai.api_key, model="gpt-4o-mini")

# Carregar o prompt do hub
try:
    prompt = hub.pull("rlm/rag-prompt")
except Exception as e:
    raise RuntimeError(f"Erro ao carregar o prompt do hub: {e}")
prompt

c:\Users\marce\OneDrive - mtegovbr\0001_NOVA_PASTA\DESENVOLVIMENTO_PESSOAL\001_mba_ciencia_dados_enap\007_ia_generativa_contexto_adm_pub\chatbot-pronaf\venv3.13\Lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [12]:
# Função para formatar os documentos recuperados
def format_docs(documentos):
    """
    Concatena os conteúdos dos documentos em um único texto com separador de duas quebras de linha.
    Adicionalmente, inclui um cabeçalho com metadados relevantes (por exemplo, número da página), se disponíveis.

    Args:
        documentos (list): Lista de objetos que possuem os atributos 'page_content' e, opcionalmente, 'metadata'.

    Returns:
        str: Texto concatenado dos conteúdos dos documentos que possuem conteúdo válido.
    """
    formatted_texts = []  # Lista para armazenar os textos formatados

    for doc in documentos:
        # Verifica se o objeto possui o atributo 'page_content'
        if hasattr(doc, "page_content"):
            content = doc.page_content.strip()  # Remove espaços extras do início e fim
            # Se o conteúdo não estiver vazio
            if content:
                header = ""
                # Se existirem metadados e, em particular, o número da página estiver disponível, insere o cabeçalho
                if hasattr(doc, "metadata") and doc.metadata.get("page"):
                    header = f"Página: {doc.metadata.get('page')}\n"
                formatted_texts.append(f"{header}{content}")

    # Junta os textos formatados usando duas quebras de linha como separador
    return "\n\n".join(formatted_texts)


########### Teste de formatação dos documentos ###########
from types import SimpleNamespace

# Suponha que a função format_docs já esteja definida, conforme a versão agregada.

# # Criando objetos de teste
# doc1 = SimpleNamespace(
#     page_content="  Conteúdo do primeiro documento.  ",
#     metadata={"page": 1}
# )

# doc2 = SimpleNamespace(
#     page_content="Conteúdo do segundo documento.",
#     metadata={"page": 2}
# )

# # Objeto com conteúdo apenas de espaços (deve ser ignorado)
# doc3 = SimpleNamespace(
#     page_content="   ",
#     metadata={"page": 3}
# )

# # Objeto sem metadados
# doc4 = SimpleNamespace(
#     page_content="Conteúdo sem metadados."
# )

# # Objeto sem o atributo page_content (deve ser ignorado)
# doc5 = SimpleNamespace(
#     metadata={"page": 5}
# )

# # Lista de documentos de teste
# documentos_teste = [doc1, doc2, doc3, doc4, doc5]

# # Testando a função format_docs
# resultado = format_docs(documentos_teste)
# print(resultado)

In [13]:

# Configurar o número de documentos a serem recuperados
n_documentos = 3

# Definir a entrada do pipeline
entrada_pipeline = {
    "question": RunnablePassthrough(),
    "context": vector_db.as_retriever(k=n_documentos) | format_docs
}

########Teste de entrada do pipeline ########
# # Testando o pass-through da questão
# dummy_question = "O que é crédito rural?"
# processed_question = RunnablePassthrough().invoke(dummy_question)
# print("Pergunta processada:", processed_question)


# # Executar a recuperação e formatação do contexto para a consulta de teste
# formatted_context = (vector_db.as_retriever(k=n_documentos) | format_docs).invoke(dummy_question)
# print("Contexto recuperado e formatado:")
# print(formatted_context)

#### Teste de recuperação de documentos ####
# docs_teste = vector_db.as_retriever(k=n_documentos).invoke(dummy_question)
# for i, doc in enumerate(docs_teste, start=1):
#     print(f"Documento {i}:\n{doc.page_content}\n")


In [14]:
# Construir o pipeline RAG
rag = (
    entrada_pipeline
    | prompt
    | llm
    | StrOutputParser()
)


In [15]:

# Testar o pipeline com uma pergunta
try:
    pergunta = "Quais programas existem?"
    resposta = rag.invoke(pergunta)
    print("Resposta do RAG:", resposta)
except Exception as e:
    raise RuntimeError(f"Erro ao executar o pipeline RAG: {e}")

Resposta do RAG: Os programas mencionados incluem o Proagro e o Programa ABC+. O Proagro cobre riscos para cooperativas e associações de produtores, enquanto o Programa ABC+ se refere a operações de investimento contratadas dentro de um período específico. Ambos têm regras e limites que devem ser respeitados.


In [16]:
prompt = "O que é MCR?"
rag.invoke(prompt)

'MCR refere-se a um conjunto de normas e condições relacionadas à exigibilidade da Poupança Rural e outras operações financeiras. Os códigos mencionados como MCR 6-2 e MCR 6-4 tratam de aspectos específicos da apuração e cumprimento das exigências regulatórias por instituições financeiras. As regras são definidas para assegurar a utilização adequada dos recursos e o controle das operações financeiras.'

In [17]:
prompt = "Quais são os programas disponíveis?"
resposta = rag.invoke(prompt)
print("Resposta do RAG:", resposta)


Resposta do RAG: Os programas disponíveis mencionados incluem o Proagro e CPR (Certificado de Recebíveis do Agronegócio). O Proagro oferece suporte financeiro para agricultores, enquanto o CPR envolve acordos de adesão e solidariedade entre beneficiários. Para mais detalhes sobre outros programas, a consulta a documentos específicos pode ser necessária.


In [15]:
prompt = "Quais são as faixas de renda e finalidades específicas?"
resposta = rag.invoke(prompt)
print("Resposta do RAG:", resposta)

Resposta do RAG: As faixas de renda não estão claramente especificadas no contexto. As finalidades mencionadas incluem implantar, utilizar e/ou recuperar projetos técnicos ou propostas para investimentos. Essas finalidades são regidas por regulamentações específicas, conforme as resoluções do CMN.


In [17]:
prompt = "O que é PRONAF?"
resposta = rag.invoke(prompt)
print("Resposta do RAG:", resposta)

Resposta do RAG: O PRONAF, ou Programa Nacional de Fortalecimento da Agricultura Familiar, é um programa brasileiro que visa apoiar e financiar a agricultura familiar por meio de condições específicas de crédito rural. Sua finalidade é promover o desenvolvimento sustentável e a inclusão social dos agricultores familiares. As normas e procedimentos relacionados ao PRONAF são estabelecidos pelo Ministério da Agricultura.


In [20]:
prompt = "Qual resolução fala sobre: Dispõe sobre as exigibilidades de aplicação em crédito rural ao amparo dos recursos obrigatórios (MCR 6-2) e da poupança rural (MCR 6-4), sobre Depósito Interfinanceiro Vinculado ao Crédito Rural (DIR), sobre a Declaração de Aptidão ao Pronaf (DAP) do Pronaf e prazos do Proagro."
resposta = rag.invoke(prompt)
print("Resposta do RAG:", resposta)

Resposta do RAG: A resolução que trata das exigibilidades de aplicação em crédito rural e outros aspectos mencionados é a Resolução CMN nº 4.901. Ela estabelece normas relativas ao uso de recursos obrigatórios e da poupança rural, além de questões sobre depósitos interfinanceiros vinculados ao crédito rural e a Declaração de Aptidão ao Pronaf. Para informações específicas, recomenda-se consultar o Manual de Crédito Rural (MCR).


In [22]:
print("entrada_pipelina", entrada_pipeline)
print("prompt", prompt)
print("llm", llm)   

entrada_pipelina {'question': RunnablePassthrough(), 'context': VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002846CF8E660>, search_kwargs={})
| RunnableLambda(format_docs)}
prompt Qual resolução fala sobre: Dispõe sobre as exigibilidades de aplicação em crédito rural ao amparo dos recursos obrigatórios (MCR 6-2) e da poupança rural (MCR 6-4), sobre Depósito Interfinanceiro Vinculado ao Crédito Rural (DIR), sobre a Declaração de Aptidão ao Pronaf (DAP) do Pronaf e prazos do Proagro.
llm client=<openai.resources.chat.completions.completions.Completions object at 0x000002842747DE80> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000002842747A270> root_client=<openai.OpenAI object at 0x00000284272DFE00> root_async_client=<openai.AsyncOpenAI object at 0x000002842747DFD0> model_name='gpt-4o-mini' model_kwargs={} openai_api_key=SecretStr('**********')


In [36]:
# Configurar o número de documentos a serem recuperados
n_documentos = 3

# Obter o objeto recuperador com o parâmetro k definido
retriever = vector_db.as_retriever(k=n_documentos)

# Executar a recuperação para uma determinada pergunta (substitua 'Sua pergunta aqui')
docs = retriever.get_relevant_documents("O que é PRONAF?")

# Exibir os documentos recuperados
for i, doc in enumerate(docs, start=1):
    print(f"Documento {i}:\n{doc.page_content}\n")

C:\Users\marce\AppData\Local\Temp\ipykernel_28440\3909283936.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("O que é PRONAF?")


InternalError: Error getting collection: Database error: error returned from database: (code: 1) no such table: collections